## Why funclift

funclift is a Python library that provides functors, applicatives, monads, profunctors, free monads and many other functional programming constructs for composing synchronous as well as asynchronous functions.

This tutorial will use toy examples to walk you through the various features of funclift for the ease of learning. For practical usages of funclift, please refer to other examples in this git repository.

## function composition

Before we dive into the details, it's helpful to point out that all these various constructs such as functors, applicatives and monads serve a common purpose: function composition. As you go through this tutorial, if you find yourself asking why a specific feature is useful, then the answer will mostly likely be related to the composition of certain types of functions.

## pure functions

There are pure and impure functions. Pure functions are like mathematical functions. They are total functions that always map the same input to the same output with no side effects. Here's an example of a pure function in Python:

In [ ]:
def foo(b: bool) -> int:
    return (2 if b else 3)

The function takes a boolean value as input. It's a total function because it's defined for all possible values of the `bool` type. If the input is True, it will always return 2. If the input is False, it will always return 3. So for the same input, it will always return the same output. Moreover, the function incurs no side effects such as performing network IO, mutating external states or throwing exceptions.

## Composing pure functions

Pure functions are easy to compose. Here's an example. Suppose we have another pure function `bar` like this:

In [ ]:
def bar(n: int) -> str:
    return 'h' * n

Because the output type of foo matches the input type of bar, we can compose the two functions like this:

In [ ]:
def foo_then_bar(b: bool) -> str:
    return bar(foo(b))

## impure functions

Okay, we just saw how straightforward it is to compose pure functions. How about impure functions? Here's an impure function in Python:

In [ ]:
def ten_mod_by(n: int) -> int:
    return 10 % n

The function is impure because it's not defined when the input is zero. If you pass zero as input to the function, it will throw ZeroDivisionError. So what should we do? Well we need to turn it into a pure function. To that end, one might be attempted to rewrite the function to something like this:

In [ ]:
def ten_mod_by(n: int) -> int | None:
    if n == 0:
        return None
    
    return 10 % n

The function indeed is a pure one since it's now defined for all inputs including zero. However, there's a critical issue with the function: it's not very composable with other functions. For example, suppose we have another function like this:

In [ ]:
def remainder_in_text(r: int) -> str:
    return 'remainder is ' + str(r)

Here's how it looks to compose `ten_mod_by` with `remainder_in_text`:

In [ ]:
def ten_mod_by_in_text(x: int) -> str | None:
    r = ten_mod_by(x)
    if r:
        return remainder_in_text(r)
    else:
        return None

The if-else statement in the code snippet above might not seem too bad. However, if we need to compose several of the similar functions together in this fashion, we will end up with deeply nested if-ease branches that are not so pleasant to read.

The solution for turning an impure partial function into a total function while retaining good composability is the Option class in funclift. Here's how we use it to rewrite the `ten_mod_by` function:

In [ ]:
from funclift.types.option import Option, Nothing, Some

def ten_mod_by(n: int) -> Option[int]:
    if n == 0:
        return Nothing()
    
    return Some(10 % n)

The function returns Nothing() if the input is zero. Otherwise, it returns the result wrapped in an instance of the Some class. Both `Nothing` and `Some` are subclasses of the `Option` class.

Here's how we compose the new `ten_mod_by` function with the `remainder_in_text` function:

In [ ]:
def ten_mod_by_in_text(x: int) -> Option[str]:
    r = ten_mod_by(x)
    return r.fmap(remainder_in_text)